In [6]:
import pandas as pd
df_test = pd.read_csv("C:/Users/U_M1P8G/Desktop/IDE/Доп. главы высшей математики/Кластеризация и техники понижения размерности/data/test.txt", 
                 sep='\s+',  header=None)
df_train = pd.read_csv("C:/Users/U_M1P8G/Desktop/IDE/Доп. главы высшей математики/Кластеризация и техники понижения размерности/data/train.txt", 
                 sep='\s+',  header=None)

df_test_labels = pd.read_csv("C:/Users/U_M1P8G/Desktop/IDE/Доп. главы высшей математики/Кластеризация и техники понижения размерности/data/test_labels.txt", 
                 sep='\s+',  header=None)
df_train_labels = pd.read_csv("C:/Users/U_M1P8G/Desktop/IDE/Доп. главы высшей математики/Кластеризация и техники понижения размерности/data/train_labels.txt", 
                 sep='\s+',  header=None)

In [5]:
df_test.head(3)

,0,1,2,3,4,5,6,7,8,9,...,551,552,553,554,555,556,557,558,559,560
0,0.257178,-0.023285,-0.014654,-0.938404,-0.920091,-0.667683,-0.952501,-0.925249,-0.674302,-0.894088,...,0.071645,-0.330370,-0.705974,0.006462,0.162920,-0.825886,0.271151,-0.720009,0.276801,-0.057978
1,0.286027,-0.013163,-0.119083,-0.975415,-0.967458,-0.944958,-0.986799,-0.968401,-0.945823,-0.894088,...,-0.401189,-0.121845,-0.594944,-0.083495,0.017500,-0.434375,0.920593,-0.698091,0.281343,-0.083898
2,0.275485,-0.026050,-0.118152,-0.993819,-0.969926,-0.962748,-0.994403,-0.970735,-0.963483,-0.939260,...,0.062891,-0.190422,-0.640736,-0.034956,0.202302,0.064103,0.145068,-0.702771,0.280083,-0.079346


In [9]:
df_test['labels'] = df_test_labels[0]
df_test['labels'].value_counts()

df_train['labels'] = df_train_labels[0]
df_train['labels'].value_counts()

labels
6    1407
5    1374
4    1286
1    1226
2    1073
3     986
Name: count, dtype: int64

In [16]:
df = pd.concat([df_train, df_test], axis=0)
df = df.reset_index(drop=True)
df.shape

(10299, 562)

In [17]:
df['labels'].value_counts()

labels
6    1944
5    1906
4    1777
1    1722
2    1544
3    1406
Name: count, dtype: int64

Далее необходимо отмасштабировать признаки. Будем использовать для этого алгоритм StandardScaler. Примените его ко всем значениям признаков и впишите в качестве ответа значение первого признака для первого объекта, предварительно округлив его до двух знаков после точки-разделителя.

In [46]:
from sklearn.preprocessing import StandardScaler
import numpy as np

X = np.concatenate((df_train, df_test))
y = np.concatenate((df_train_labels[0].values, df_test_labels[0].values))
y.shape

(10299,)

Пора переходить к кластеризации. Для начала определите оптимальное количество кластеров, используя внутренние меры кластеризации. Используйте все известные вам коэффициенты, реализуемые в библиотеке sklearn: коэффициент силуэта, индекс Калински — Харабаса и индекс Дэвиса — Болдина. В качестве алгоритма возьмите k-means++, в качестве значения параметра random_state — число 42.

Выведите оптимальное количество кластеров для каждой метрики, перебирая значения от 2 до 9 включительно.

In [35]:
#определяем скейлер
scaler = StandardScaler()
#масштабируем данные
X_scaled = scaler.fit_transform(X)
#выводим нужное значение
X_scaled[0][0]

np.float64(0.21053385417675513)

In [39]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.metrics import calinski_harabasz_score 
from sklearn.metrics import davies_bouldin_score

for i in range(2,10):
	# обучаем модель k-means с текущим количеством кластеров
    km = KMeans(n_clusters = i , random_state = 42 ).fit(X_scaled)
	#предсказываем метки
    y_pred = km.labels_
	#выводим результат для коэффициента силуэта
    print (i, silhouette_score(X_scaled,  y_pred))
for i in range(2,10):
    km = KMeans(n_clusters = i , random_state = 42 ).fit(X_scaled)
    y_pred= km.labels_
    print (i, calinski_harabasz_score(X_scaled,y_pred))
for i in range(2,10):
    km = KMeans(n_clusters = i , random_state = 42 ).fit(X_scaled)
    y_pred = km.labels_
    print (i, davies_bouldin_score(X_scaled, y_pred))
    

2 0.3942991642725773
3 0.31694926882659896
4 0.27861040757908057
5 0.2600378262425835
6 0.11688980679131775
7 0.10109286313257404
8 0.08367304664253887
9 0.08223835107662579
2 7900.2459276444215
3 5044.559295246528
4 3674.551083222876
5 2850.6870801648743
6 2508.8763459252123
7 2174.68376339336
8 1886.1697492493126
9 1772.0057691635127
2 1.0695019794401763
3 1.7833526179028067
4 2.0465455780310697
5 2.266431849948252
6 2.5269266958046734
7 2.648445147386934
8 2.77193032799534
9 2.6300947180472125


Теперь давайте оценим качество кластеризации с точки зрения внешних мер. Реализуйте кластеризацию с помощью классической версии алгоритма k-means. Пусть количество кластеров будет соответствовать количеству активностей. Задайте следующие значения параметров: random_state=42, init='random'.

In [48]:
from sklearn.metrics import homogeneity_score
from sklearn.metrics import completeness_score
from sklearn.metrics import adjusted_rand_score

model= KMeans(n_clusters=6, init='random', random_state=42)
#обучаем модель
model.fit(X_scaled)
#вычисляем значение однородности
print(homogeneity_score(y, model.labels_))
#вычисляем значение полноты
print(completeness_score(y, model.labels_))
#вычисляем значение скорректированного индекса Рэнда
print(adjusted_rand_score(y, model.labels_))
  

0.5417556970823177
0.5805397962855482
0.42072733220323044


In [49]:
#создаём таблицу сопряжённости
ct = pd.crosstab(df['labels'], model.labels_)
#определяем название активностей
ct.index = ['ходьба', 'подъём', 
            'спуск', 'сидение', 'стояние', 'лежание']
ct.columns = list(range(1,7))

print("Количество наблюдений по активностям в кластерах:")
print(ct)

Количество наблюдений по активностям в кластерах:
           1     2    3     4    5     6
ходьба   741   897    0     0   84     0
подъём   297  1236    2     0    9     0
спуск    883   309    0     0  214     0
сидение    0     1  443    88    0  1245
стояние    0     0  559     0    0  1347
лежание    0     4  329  1558    0    53


In [56]:
ct = pd.crosstab(df['labels'], model.labels_)
ct

col_0,0,1,2,3,4,5
labels,,,,,,
1,741,897,0,0,84,0
2,297,1236,2,0,9,0
3,883,309,0,0,214,0
4,0,1,443,88,0,1245
5,0,0,559,0,0,1347
6,0,4,329,1558,0,53


In [57]:
model= KMeans(n_clusters=2, init='random', random_state=42)
#обучаем модель
model.fit(X_scaled)
#вычисляем значение однородности
print(homogeneity_score(y, model.labels_))
#вычисляем значение полноты
print(completeness_score(y, model.labels_))
#вычисляем значение скорректированного индекса Рэнда
print(adjusted_rand_score(y, model.labels_))

0.37797082246896085
0.9795305596996312
0.3296106738894193


In [58]:
ct = pd.crosstab(df['labels'], model.labels_)
ct

col_0,0,1
labels,,
1,1722,0
2,1536,8
3,1406,0
4,3,1774
5,0,1906
6,12,1932


In [59]:
from sklearn.cluster import AgglomerativeClustering

agg_model = AgglomerativeClustering(n_clusters=2)
agg_labels = agg_model.fit_predict(X_scaled)

agg_completeness = completeness_score(y, agg_labels)
print(f"Агломеративная - полнота: {agg_completeness:.3f}")

Агломеративная - полнота: 1.000
